In [1]:
import wandb
from data_loaders import make_train_data, make_test_val_data, make_model
from train import train, test, precision, recall, f1_score
import torch
import time
import gc
import config_file
from clean_data import load_all_houses_with_device
import random
import optuna
from data_loaders import PecanStreetDataset
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from lstm_seq2point import LSTM
from clean_data import normalize_y
from sklearn import metrics
import pandas as pd
from scipy import stats
from matplotlib.lines import Line2D
import matplotlib.transforms

In [2]:
wandb.login()

print(torch.cuda.is_available())

gc.collect()
torch.cuda.empty_cache()

print(torch.__version__)

In [3]:
config_ = config_file.load_hyperparameters("refrigerator1")

In [4]:
homes = load_all_houses_with_device(config_file.path, config_['appliance'])

In [5]:
def model_pipeline(hyperparameters, train_months, test_month, appliance, window_length, train_buildings,
                   test_buildings, patience):
    with wandb.init(project="global_models_feb10", config=hyperparameters):
        wandb.run.name = str(config_['appliance'])+"_Test:"+str(test_buildings)+"_Train:" + str(train_buildings)

        config = wandb.config

        # lengths = [85320, 132480, 132480, 132480, 132480, 132480, 132480]

        model, criterion, optimizer = make_model(config)

        print(model)
        print("Window Length: ", window_length)

        wandb.watch(model, criterion, log="all", log_freq=10)

        example_ct = 0
        batch_ct = 0
        all_epochs = 0

        #Scheduler for training on single building
#         scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.85, verbose=True)
        
        #base_lr: 0.001*lr, max_lr: 4*lr, step_size_up:50, step_size_down:2000
        scheduler = torch.optim.lr_scheduler.CyclicLR(
            optimizer,
            base_lr = 2*config_['learning_rate'], #0.01
            max_lr = 10*config_['learning_rate'], #1
            step_size_up = 100, #200
            step_size_down = 500, #1000
            gamma = 0.9, #1
            cycle_momentum=False,
            verbose=True
        )

        validation_loader, test_loader, test_val_seq_min, test_val_seq_max = make_test_val_data(
            config,
            test_month,
            appliance,
            window_length,
            test_buildings
        )

        time_log = time.time()
        train_loader, train_seq_min, train_seq_max = make_train_data(
            config,
            train_months,
            appliance,
            window_length,
            train_buildings
        )
        
        model, example_ct, batch_ct, all_epochs, best_model = train( 
            model,
            train_loader,
            validation_loader,
            criterion,
            optimizer,
            config,
            example_ct,
            batch_ct,
            all_epochs,
            scheduler,
            test_val_seq_min,
            test_val_seq_max,
            train_seq_min,
            train_seq_max,
            patience
        )

        print("Time to train on one home: ", time.time() - time_log)

        results = test(best_model, test_loader, criterion, test_val_seq_min, test_val_seq_max)

    return model, results, best_model

In [6]:
home_ids = homes.dataid.unique()

In [7]:
len(home_ids)

38

In [8]:
home_ids

array([  142,   183,   335,   387,   526,  1240,  1417,  2126,  2358,
        3383,  3488,  3976,  3996,  5058,  5192,  6069,  6178,  6240,
        6526,  6564,  6594,  6672,  6703,  7069,  7365,  7935,  8627,
        8825,  9002,  9004,  9053,  9290, 10182, 10811, 10983, 11421,
       11785, 11878], dtype=int64)

In [9]:
random_select = [2,3,4,5,6,7,8,9,10,15,20,25,30,35,38]
#random_select = [len(home_ids)]
#random_select = [53]

In [10]:
PATH = r"C:\Users\aar245.CORNELL\Desktop\privacy_preserving_nn\models_squ_foot_filter"

In [11]:
train_homes_from_fl = [
    [5058, 2358, 8825, 10811, 526],
    [142, 10811, 2358, 6240, 2126, 1417, 7365, 5058, 10182, 9004],
    [9053, 2561, 2126, 7021, 526, 9973, 6178, 3700, 8825, 142, 7069, 335, 690, 6526, 183],
    [183, 6240, 5058, 6672, 8825, 3700, 7365, 9053, 3976, 9973, 6526, 3996, 526, 3488, 387, 11878, 7021, 10811, 9290, 6178],
    [6672, 9290, 6178, 10983, 3700, 6526, 3488, 6240, 9973, 3976, 9004, 2126, 3383, 142, 2358, 690, 7021, 387, 10182, 10811, 5058, 8825, 2561, 526, 3996],
    [526, 10811, 9973, 7021, 5058, 335, 3488, 9053, 10983, 3976, 11878, 142, 2561, 8825, 6672, 387, 183, 6178, 1417, 9290, 10182, 690, 2358, 7365, 3383, 6240, 7069, 9004, 3996, 3700],
    [5058, 2358, 3488, 10182, 6672, 11878, 7021, 6526, 335, 10164, 142, 9973, 10983, 6240, 2126, 1417, 7069, 6178, 8825, 3383, 9004, 387, 3976, 3700, 9290, 3996, 690, 7365, 9053, 183, 2561, 526, 10811]
]

In [12]:
test_homes_from_fl = [526, 5058, 526, 3488, 10983, 7021, 2358]

In [13]:
final_results = {}
random.seed(3)
train_homes = []
best_models = []
max_patience = 200
min_patience = 50

for i in home_ids:
    gc.collect()
    torch.cuda.empty_cache()
    training_homes=[i]
    #training_homes = random.sample(list(home_ids), k=1)
    #training_homes = train_homes_from_fl[-1]
    testing_homes = [random.choice(training_homes)]
    #testing_homes = [test_homes_from_fl[-1]]
    #patience = int((max_patience-min_patience)/(1-random_select[-1])*len(training_homes)+max_patience+(max_patience-min_patience)/(1-random_select[-1]))
    patience = 2000
    print("patience: ", patience)
    print("training_home: ", training_homes)
    print("test_home: ", testing_homes)
    model, per_house_result, best_model = model_pipeline(
    config_,
    'sept_oct_nov',
    'dec',
    config_['appliance'],
    config_['window_size'],
    training_homes,
    testing_homes,
    patience)
    result = {str(config_["appliance"])+"_Train_home_"+str(training_homes)+"_Test_home_"+str(testing_homes)+"_total_homes_"+str(1): per_house_result}
    final_results.update(result)
    print(final_results)
    #model.cpu()
    #torch.save(model.state_dict(), PATH+"\\refrigerator_model_total_houses_"+str(random_select[i])+"_trial_3.pth")
    best_model.cpu()
    model.cpu()
    best_models.append(best_model)